In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib

In [ ]:
df4 = pd.read_csv('/kaggle/input/testset/april_comments.csv')
df3 = pd.read_csv('/kaggle/input/testset/march_comments.csv')
df1 = pd.read_csv('/kaggle/input/testset/january_comments.csv')
df2 = pd.read_csv('/kaggle/input/testset/february_comments.csv')
topics = pd.read_csv('/kaggle/input/topic907/topics.csv')
#sentiments = pd.read_csv('/kaggle/input/topic907/sentiment912.csv')

frame = pd.concat([df1, df2, df3, df4], ignore_index=True)
frame[['date','author','preview']]

result = pd.merge(frame[['date','author','preview']],
                   topics[['topic1']],
                   how = 'inner',
                   left_index=True, right_index=True)

result.rename(columns={ "topic1": "topics"}, inplace=True)
result

In [ ]:
result.index.name = 'id'
result['date']= result['date'].astype('datetime64[ns]')
df = result

topic1 = df.loc[df['topics'] == 1]
topic1_num = topic1.groupby('date')['topics'].count().reset_index()
topic1_num.rename(columns ={'topics': 'topic1_num'}, inplace=True )

topic2 = df.loc[df['topics'] ==2]
topic2_num = topic2.groupby('date')['topics'].count().reset_index()
topic2_num.rename(columns ={'topics': 'topic2_num'}, inplace=True )
topic2_num

topic3 = df.loc[df['topics'] ==3]
topic3_num = topic3.groupby('date')['topics'].count().reset_index()
topic3_num.rename(columns ={'topics': 'topic3_num'}, inplace=True )
topic3_num

topic4 = df.loc[df['topics'] ==4]
topic4_num = topic4.groupby('date')['topics'].count().reset_index()
topic4_num.rename(columns ={'topics': 'topic4_num'}, inplace=True )
topic4_num

topic5 = df.loc[df['topics'] == 5]
topic5_num = topic5.groupby('date')['topics'].count().reset_index() 
topic5_num.rename(columns ={'topics': 'topic5_num'}, inplace=True )

topic12 = pd.merge(topic1_num,
                topic2_num,
                how = 'outer',
                on = 'date')

topic123 = pd.merge(topic12,
                topic3_num,
                how = 'outer',
                on = 'date')

topic1234 = pd.merge(topic123,
                topic4_num,
                how = 'outer',
                on = 'date')

topic12345 = pd.merge(topic1234,
                topic5_num,
                how = 'outer',
                on = 'date')

topic12345['topic5_num'] = topic12345['topic5_num'].fillna(0.0)
topic12345['topic1_num'] = topic12345['topic1_num'].fillna(0.0)
topic12345['topic2_num'] = topic12345['topic2_num'].fillna(0.0)
topic12345['topic3_num'] = topic12345['topic3_num'].fillna(0.0)
topic12345['topic4_num'] = topic12345['topic4_num'].fillna(0.0)
topic12345.rename(columns ={'topic1_num':'topic1','topic2_num':'topic2','topic3_num':'topic3','topic4_num':'topic4','topic5_num':'topic5'}, inplace = True)
topic12345 = topic12345.sort_values(by='date',ascending=True)
topic12345

In [ ]:
topic12345.to_csv('counttopics913.csv', index = False)

----
# A tutorial of machine learning mastery
## ARIMA (08-25)
[https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/](http://)


In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
 
def parser(x):
    return datetime.strptime( x, '%Y-%m-%d')
 
series = read_csv('/kaggle/working/counttopics913.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
print(series.head())
series.plot()
pyplot.show()

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
 
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')
 
series = read_csv('/kaggle/working/counttopics913.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
autocorrelation_plot(series)
pyplot.show()

In [ ]:
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
 
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')
 
series = read_csv('/kaggle/working/counttopics913.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# fit model
model = ARIMA(series, order=(1,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())

In [ ]:
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')

series = read_csv('/kaggle/working/counttopics913.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

-----
## modelling

In [ ]:
!pip install livelossplot

In [ ]:

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten,LSTM,RepeatVector,TimeDistributed,Conv1D,MaxPooling1D
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from livelossplot.keras import PlotLossesCallback

from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df = topic1_num
df.rename(columns={'topic1_num': 'x'}, inplace=True) 
df = df.drop(['date'],axis = 1)

#df = topic1_num
#df.rename(columns={'topic1_num': 'x'}, inplace=True) 
#df = df.drop(['date'],axis = 1)

#df = topic2_num
#df.rename(columns={'topic2_num': 'x'}, inplace=True) 
#df = df.drop(['date'],axis = 1)

#df = topic3_num
#df.rename(columns={'topic3_num': 'x'}, inplace=True) 
#df = df.drop(['date'],axis = 1)

#df = topic4_num
#df.rename(columns={'topic4_num': 'x'}, inplace=True) 
#df = df.drop(['date'],axis = 1)

In [ ]:
## use the past days to forecast the following.
def train_test_builder(df,size_test=24,length_train_sequence = 24,length_forecast = 12, normalize = False):
    
    train_df = df[:-size_test]    
    
    # create training sets
    train_x, train_y = window_splitter(train_df)
    
    # create test set
    tmp = df[:length_forecast]
    tmp = pd.concat([df,tmp],ignore_index = True)
    tmp[-length_forecast:] = np.nan  # unknown
    test_x,test_y = window_splitter(tmp)
    
    # drop elements in training
    test_x = test_x[train_x.shape[0]:]
    test_y = test_y[train_x.shape[0]:]
    for i in range(length_forecast):
        test_y[i,:(11-i)]=np.nan # present in training_set
    
    # normalize
    if normalize:
        m = train_df.x.mean()
        sd = train_df.x.std()
        train_x -= m
        train_x/= sd
        test_x -=m
        test_x /= sd
    
    return train_x,train_y,test_x,test_y

# iterate over the time steps and divide the data into overlapping windows; 
def window_splitter(train_df,length_train_sequence = 24,length_forecast = 12):
    i = 0
    x,y = [],[]
    while i + length_train_sequence+ length_forecast < len(train_df):
        x.append(train_df.x[i:(i+length_train_sequence)].values)
        y.append(train_df.x[(i+length_train_sequence):(i+length_train_sequence+length_forecast)].values)
        i+=1

    x = np.array(x).reshape(-1,length_train_sequence,1)
    y = np.array(y).reshape(-1,length_forecast)
    
    return x,y

In [ ]:
# a naive seasonal decomposition
series = df.x.values
result = seasonal_decompose(series, model='additive',freq=12,two_sided = False,)
plt.plot(result.trend)
plt.plot(result.seasonal)
plt.plot(result.resid)
plt.plot(result.observed)

In [ ]:
deseasonalised_df = pd.DataFrame({'x':result.trend[12:]+result.resid[12:]})
deseasonalised_df.head()

In [ ]:
# build both versions
des_train_x,des_train_y,des_test_x, _ = train_test_builder(deseasonalised_df)
train_x,train_y,test_x,test_y = train_test_builder(df)

train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
# evaluation function
def evaluate_predictions(pred_y, test_y):
    return pd.DataFrame(abs(test_y-pred_y)).mean(skipna = True)


def plot_evaluation(pred_y,test_y,n=12):
    scores = evaluate_predictions(pred_y,test_y)

    print("Mean absolute error test set:",scores.mean())

    plt.figure(figsize=(6,4))
    plt.plot(np.arange(1,13),scores)
    plt.xticks(np.arange(1,13))
    plt.xlabel("horizon [months]", size = 15)
    plt.ylabel("MAE", size = 15)
    plt.title("Scores LSTM on test set")
    plt.show()

    plt.figure(figsize=(6,4))
    plt.title("LSTM forecasting - test set window")
    plt.plot(np.arange(1,13),pred_y[n:(n+1)].reshape(-1,1),label = "predictions")
    plt.plot(np.arange(1,13),test_y[n:(n+1)].reshape(-1,1),label = "true values")
    plt.xticks(np.arange(1,13))
    plt.xlabel("horizon [months]", size = 15)
    plt.legend()
    plt.show()

## LSTM

In [ ]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

# define model
model = Sequential()
model.add(LSTM(256, activation='relu', input_shape=(None, 1)))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(n_outputs))

# compile
model.compile(loss='mse', optimizer= Adam(lr=1e-3), metrics = ["mae"])

# callbacks
early_stopping = EarlyStopping(patience=32, monitor='val_loss', mode='auto', restore_best_weights=True)
callbacks=[PlotLossesCallback(), early_stopping]

# fit network
model.fit(des_train_x, des_train_y, 
          validation_split = 0.2, 
          epochs=100, 
          shuffle = True,
          batch_size=16,
          verbose=1,
       #   callbacks = callbacks
         )

In [ ]:
# predict test set
pred_y = model.predict(des_test_x)
# add seasonality of the past year
for i in np.arange(pred_y.shape[0]-1,-1,-1):
    pred_y[pred_y.shape[0]-1-i] += result.seasonal[-(13+i):-(1+i)]

In [ ]:
plot_evaluation(pred_y,test_y,n=12)

In [ ]:
# store predictions on test set
tmp = pd.DataFrame(test_y-pred_y)
tmp.columns = np.arange(1,13)

tmp.to_csv("lstm_predictions.csv",index = False)

------
## Encoder Decoder 


In [ ]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

des_train_y = des_train_y.reshape((des_train_y.shape[0], des_train_y.shape[1], 1))
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.1))
model.add(RepeatVector(n_outputs))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(1)))

# compile
model.compile(loss='mse', optimizer= Adam(lr=1e-3), metrics = ["mae"])

# callbacks
early_stopping = EarlyStopping(patience=16, monitor='val_loss', mode='auto', restore_best_weights=True)
callbacks=[PlotLossesCallback(), early_stopping]


# fit network
model.fit(des_train_x, des_train_y, 
          validation_split = 0.1, 
          epochs=100, 
          shuffle = True,
          batch_size=16,
          verbose=1,
     #     callbacks = callbacks,
         )


# predict test set
pred_y = model.predict(des_test_x).reshape(-1,12)
# add seasonality of the past year
for i in np.arange(pred_y.shape[0]-1,-1,-1):
    pred_y[pred_y.shape[0]-1-i] += result.seasonal[-(13+i):-(1+i)]

plot_evaluation(pred_y,test_y,n=12)

---
## Encoder Decoder CNN LSTM 

In [ ]:
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

des_train_y = des_train_y.reshape((des_train_y.shape[0], des_train_y.shape[1], 1))
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.1))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(RepeatVector(n_outputs))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(TimeDistributed(Dense(1)))
# compile
model.compile(loss='mse', optimizer= Adam(lr=1e-3), metrics = ["mae"])

# callbacks
early_stopping = EarlyStopping(patience=16, monitor='val_loss', mode='auto', restore_best_weights=True)
callbacks=[PlotLossesCallback(), early_stopping]

# fit network
model.fit(des_train_x, des_train_y, 
          validation_split = 0.1, 
          epochs=100, 
          shuffle = True,
          batch_size=16,
          verbose=1,
        #  callbacks = callbacks,
         )

# predict test set
pred_y = model.predict(des_test_x).reshape(-1,12)
# add seasonality of the past year
for i in np.arange(pred_y.shape[0]-1,-1,-1):
    pred_y[pred_y.shape[0]-1-i] += result.seasonal[-(13+i):-(1+i)]

plot_evaluation(pred_y,test_y,n=12)



----
# A tutorial of machine learning mastery
## LSTM Network for Regression (08-24)

### [Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
Unlike regression predictive modeling, time series also adds the complexity of a sequence dependence among the input variables.

A powerful type of neural network designed to handle sequence dependence is called recurrent neural networks.



In [ ]:
import matplotlib.pyplot as plt
#topic12345.set_index('date', inplace=True)
plt.figure(figsize=(20, 10))
#plt.plot(topic12345['topic1'])
#plt.plot(topic12345['topic2'])
plt.plot(topic12345['topic3'])
#plt.plot(topic12345['topic4'])
#plt.plot(topic12345['topic5'])

plt.show()

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
topic3_num.set_index('date', inplace=True)
topic3_num = topic3_num.astype('float32')

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(topic3_num)


In [ ]:
# split into train and test sets
train_size = int(len(dataset)*0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

The ``look_back``, which is the number of previous time steps to use as input variables to predict the next time period — in this case defaulted to 1.
### look_back =1

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
dataset= numpy.reshape(dataset, (dataset.shape[0], 1, dataset.shape[1]))
yhat = model.predict(dataset)

In [ ]:
len(yhat)

In [ ]:
plt.plot(yhat)
plt.show()

----
## 2. LSTM for Regression Using the Window Method

Recent time steps can be used to make the prediction for the next time step. This is called a window, and the size of the window is a parameter that can be tuned for each problem.

Increasing the look_back argument from 1 to 3. When phrased as a regression problem, the input variables are t-2, t-1, t and the output variable is t+1.

### look_back = 3

In [ ]:
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

---
## 3. LSTM for Regression with Time Steps

Instead of phrasing the past observations as separate input features, we can use them as time steps of the one input feature, which is indeed a more accurate framing of the problem.

In [ ]:
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

## 4. LSTM with Memory Between Batches

In [ ]:
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
# create and fit the LSTM network
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(100):
    model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()
# make predictions
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
# predict test set
pred_y = model.predict(des_test_x).reshape(-1,12)
# add seasonality of the past year
for i in np.arange(pred_y.shape[0]-1,-1,-1):
    pred_y[pred_y.shape[0]-1-i] += result.seasonal[-(13+i):-(1+i)]

plot_evaluation(pred_y,test_y,n=12)

-------
# ARIMA
ARIMA stands for Auto Regressive Integrated Moving Average. While exponential smoothing models were based on a description of trend and seasonality in data, ARIMA models aim to describe the correlations in the time series. The video below explains ARIMA very well:

## hyperparameters tuning:
The complete procedure for evaluating a grid of ARIMA hyperparameters is listed below.

In [ ]:
topic12345['topic3']

In [ ]:
# tune ARIMA model for topic0

import warnings
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(a, arima_order):   #a=X
    # prepare training dataset
    train_size = int(len(a) * 0.70)
    train, test = a[0:train_size], a[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    return error

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    mse = evaluate_arima_model(dataset, order)
                    if mse < best_score:
                        best_score, best_cfg = mse, order
                    print('ARIMA%s MSE=%.3f' % (order,mse))
                except:
                    continue
    print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))


topic1 Best ARIMA(2, 0, 1) MSE=716.429

topic2 Best ARIMA(0, 1, 1) MSE=4364.329

topic3 Best ARIMA(4, 1, 2) MSE=9671.969

topic4 Best ARIMA(0, 1, 1) MSE=3458.092

topic5 Best ARIMA(2, 0, 0) MSE=8961.120

In [ ]:
# load dataset
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')
series = topic12345['topic5']
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values) 

In [ ]:
series = topic12345['topic5']
from matplotlib import pyplot
X = series.values
size = int(len(X) * 0.70)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
	model = ARIMA(history, order=(2, 0, 0))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
from pandas import read_csv
from statsmodels.tsa.arima_model import ARIMA
import numpy
 
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return numpy.array(diff)
 
# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]
  
# seasonal difference
X = series.values
days_in_month = 10
differenced = difference(X, days_in_month)
# fit model
model = ARIMA(differenced, order=(2, 0, 0) )
model_fit = model.fit(disp=0)
# multi-step out-of-sample forecast
start_index = len(differenced)
end_index = start_index + 30
forecast = model_fit.predict(start=start_index, end=end_index)
# invert the differenced forecast to something usable
history = [x for x in X]
day = 1

for yhat in forecast:
	inverted = inverse_difference(history, yhat, days_in_month)
	print('Day %d: %f' % (day, inverted))
	history.append(inverted)
	day += 1


In [ ]:
len(history)

In [ ]:
# create an array for prediction 
rng = pd.date_range('2020-04-18', periods=31, freq='D')
extenddf = pd.DataFrame({ 'date': rng, 'topic5' : history[106:len(history)]}) 
extenddf.set_index('date', inplace=True)

In [ ]:
prediction31 = pd.merge(prediction31,
                       extenddf,
                       how = 'outer',
                       on = 'date')

#prediction31 = prediction31.drop(['anger_y','love_y','enthusiasm_y'],axis = 1)
#prediction31.rename(columns={'anger_x': 'anger','love_x':'love','enthusiasm_x':'enthusiasm'}, inplace=True) 
#prediction31 = prediction31.abs()
prediction31

In [ ]:
prediction31.to_csv('predtopics913.csv',index = False)

In [ ]:
from pandas import DataFrame
prediction31 = DataFrame(extenddf,columns=['topic3'])
prediction31.index= extenddf.index.astype('datetime64[ns]')
prediction31

-----
## previous work 

In [ ]:
y1 = topic4_num['topic4_num']
y2 = topic4_num['topic4_num'].resample('MS').mean()
y1.plot(figsize=(15, 6))
y2.plot(figsize=(15, 6))
plt.ylabel('Number of topic 4 documents')
plt.xlabel('Timeline ')
plt.show()

In [ ]:
y3 = topic3_num['topic3_num']
y4 = topic3_num['topic3_num'].resample('MS').mean()
y3.plot(figsize=(15, 6))
y4.plot(figsize=(15, 6))
plt.ylabel('Number of topic 3 documents')
plt.xlabel('Timeline ')
plt.show()

In [ ]:
y5 = topic2_num['topic2_num']
y6 = topic2_num['topic2_num'].resample('MS').mean()
y5.plot(figsize=(15, 6))
y6.plot(figsize=(15, 6))
plt.ylabel('Number of topic 2 documents')
plt.xlabel('Timeline ')
plt.show()

In [ ]:
y7 = topic1_num['topic1_num']
y8 = topic1_num['topic1_num'].resample('MS').mean()
y7.plot(figsize=(15, 6))
y8.plot(figsize=(15, 6))
plt.ylabel('Number of topic 1 documents')
plt.xlabel('Timeline ')
plt.show()

In [ ]:
y9 = topic0_num['topic0_num']
y10 = topic0_num['topic0_num'].resample('MS').mean()
y9.plot(figsize=(15, 6))
y10.plot(figsize=(15, 6))
plt.ylabel('Number of topic 0 documents')
plt.xlabel('Timeline ')
plt.show()

In [ ]:
topic0_num = topic0_num.set_index('date')
y0 = topic0_num['topic0_num']

topic1_num = topic1_num.set_index('date')
y1 = topic1_num['topic1_num']

topic2_num = topic2_num.set_index('date')
y2 = topic2_num['topic2_num']

topic3_num = topic3_num.set_index('date')
y3 = topic3_num['topic3_num']

topic4_num = topic4_num.set_index('date')
y4 = topic4_num['topic4_num']

y0.plot(figsize=(15, 6))
y1.plot(figsize=(15, 6))
y2.plot(figsize=(15, 6))
y3.plot(figsize=(15, 6))
y4.plot(figsize=(15, 6))
plt.ylabel('Number of Reddit Comments')
plt.xlabel('Timeline ')
plt.show()

In [ ]:
df01 = pd.merge(topic0_num,
                    topic1_num,
                    how = 'outer',
                    on = 'date')
df012 = pd.merge(df01,
                    topic2_num,
                    how = 'outer',
                    on = 'date')
df0123 = pd.merge(df012,
                    topic3_num,
                    how = 'outer',
                    on = 'date')
df01234 = pd.merge(df0123,
                    topic4_num,
                    how = 'outer',
                    on = 'date')

In [ ]:
df01234['topic0_num'] = df01234['topic0_num'].fillna(0.0)
df01234['topic1_num'] = df01234['topic1_num'].fillna(0.0)
df01234['topic2_num'] = df01234['topic2_num'].fillna(0.0)
df01234['topic3_num'] = df01234['topic3_num'].fillna(0.0)
df01234['topic4_num'] = df01234['topic4_num'].fillna(0.0)

In [ ]:
df01234.to_csv('topics_numbers')

---
## train/Val split 
* moving average 

In [ ]:
# topic0
train_dataset = df01234[-106:-30]
val_dataset = df01234[-30:]

In [ ]:
predictions = []
for i in range(len(val_dataset.columns)):
    if i == 0:
        predictions.append(np.mean(train_dataset[train_dataset.columns[-30:]].values, axis=1))
    if i < 31 and i > 0:
        predictions.append(0.5 * (np.mean(train_dataset[train_dataset.columns[-30+i:]].values, axis=1) + \
                                  np.mean(predictions[:i], axis=0)))
    if i > 31:
        predictions.append(np.mean([predictions[:i]], axis=1))
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_avg = np.linalg.norm(predictions[:3] - val_dataset.values[:3])/len(predictions[0])

In [ ]:
len(predictions)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[0].values, marker=dict(color="dodgerblue"),
               name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[0].values, mode='lines', marker=dict(color="darkorange"),
               name="Val"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70), mode='lines', y=train_dataset.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=val_dataset.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(70, 100), y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Moving average")
fig.show()